In [32]:
import numpy
import numpy as np
import pandas as pd
import string
import nltk
import re
import matplotlib.pyplot as plt
import gensim.corpora as corpora
import gensim
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.models import Phrases, CoherenceModel
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
nltk.download('wordnet')
nltk.download('punkt')

wordnet_lemmatizer = WordNetLemmatizer()
wpt = nltk.WordPunctTokenizer()
lemmatizer = nltk.WordNetLemmatizer()

# Point to directories 
jobs = pd.read_csv(r"C:\Users\shahb\Desktop\capstone\all_jobs.csv")
resumes = pd.read_csv(r"C:\Users\shahb\Desktop\capstone\all_resumes.csv")

# How many jobs and resumes
print("Number of jobs: " + str(jobs.shape[0]))
print("Number of resumes: " + str(resumes.shape[0]))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shahb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shahb\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shahb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Number of jobs: 20000
Number of resumes: 29783


In [33]:
type(jobs)
jobs

,Unnamed: 0,job_title,category,company_name,inferred_city,inferred_state,inferred_country,job_description,job_type
0,0,Sr. Software Developer,architecture and engineering,Aerotek,Raleigh,North carolina,Usa,The chosen Sr. Software Developer will be part...,Full Time
1,1,C# Lead Software Developer,NaN,3coast,Middletown,New jersey,Usa,Position: C# Lead Software Developer Locat...,Full Time
2,2,Senior Software Developer,NaN,s.com,Hoboken,New jersey,Usa,"Senior Software Developer Hoboken, NJ Starts a...",Contract
3,3,Senior Software Developer,NaN,"Mitchell Martin, Inc.",Hoboken,New jersey,Usa,"Our client, a multinational publishing and edu...",Contract
4,4,C# Lead Software Developer,NaN,3coast,Philadelphia,Pennsylvania,Usa,Position: C# Lead Software Developer Locat...,Full Time
...,...,...,...,...,...,...,...,...,...
19995,10002,Data Scientist,Data Scientist,Upstart,Columbus,Wisconsin,United states,Join Upstart and help build the future of cred...,Full Time
19996,10003,"Data Scientist, (Staff or Principal)",Data Scientist,Realtor.com,Santa clara,New mexico,United states,"Data Scientist, (Staff or Principal) at realto...",Full Time
19997,10004,"Siri - Embedded Data Scientist, Data Organization","Data Scientist, Embedded Developer",Apple Inc.,Pittsburgh,Pennsylvania,United states,Job Summary: Would you like to play a part in ...,Full Time
19998,10005,"Data Scientist, (Staff or Principal)",Data Scientist,Realtor.com,Santa clara,New mexico,United states,"Data Scientist, (Staff or Principal) at realto...",Full Time


In [34]:
# jobs
city, citycount = np.unique(list(jobs.loc[:, "inferred_city"]), return_counts = True)
state, statecount  = np.unique(list(jobs.loc[:, "inferred_state"]), return_counts = True)
country, countrycount = np.unique(list(jobs.loc[:, "inferred_country"]), return_counts = True)

In [35]:
cities_dict = dict(zip(city, citycount))

In [36]:
# len(city) 1254
cities_dict

{'Aberdeen': 30,
 'Abilene': 1,
 'Abingdon': 2,
 'Abita springs': 1,
 'Absecon': 1,
 'Acton': 5,
 'Adamsville': 1,
 'Addison': 20,
 'Adrian': 1,
 'Agoura hills': 1,
 'Aiken': 2,
 'Akron': 5,
 'Alafaya': 2,
 'Alameda': 3,
 'Albany': 12,
 'Albemarle': 2,
 'Albuquerque': 13,
 'Alexandria': 104,
 'Alhambra': 2,
 'Aliso': 4,
 'Aliso viejo': 17,
 'Allen': 3,
 'Allentown': 6,
 'Aloha': 1,
 'Alpharetta': 73,
 'Altamonte springs': 3,
 'Amarillo': 1,
 'Ambler': 6,
 'American fork': 2,
 'Ames': 6,
 'Ames lake': 1,
 'Amherst': 2,
 'Anaheim': 1,
 'Anchorage': 1,
 'Anderson': 2,
 'Andover': 16,
 'Angeles': 5,
 'Ann arbor': 38,
 'Annandale': 4,
 'Annapolis': 99,
 'Apopka': 1,
 'Apple valley': 3,
 'Appleton': 10,
 'Arabi': 1,
 'Arden hills': 1,
 'Arizona city': 1,
 'Arlington': 201,
 'Arlington heights': 1,
 'Armonk': 15,
 'Arnold': 4,
 'Asbury': 2,
 'Ashburn': 16,
 'Asheville': 25,
 'Ashland': 2,
 'Athens': 9,
 'Atlanta': 320,
 'Atlantic city': 3,
 'Auburn': 8,
 'Auburn hills': 4,
 'Audubon': 1,
 'Au

In [38]:
city_entered = input("Enter your city ")
for key,val in cities_dict.items():
  if key.lower() == city_entered.lower():
    new_corpus  = jobs[jobs["inferred_city"] == key]

new_corpus

Enter your city Albany


,Unnamed: 0,job_title,category,company_name,inferred_city,inferred_state,inferred_country,job_description,job_type
82,82,Software Developer,Manufacturing/Mechanical,IPNS,Albany,Georgia,Usa,Read what people are saying about working here...,Undefined
1859,1859,Software Developer (Secret Clearance),Computer/Internet,MetroStar Systems,Albany,Georgia,Usa,Read what people are saying about working here...,Undefined
2850,2850,Software Developer,Computer/Internet,"CGI Group, Inc.",Albany,Georgia,Usa,Read what people are saying about working here...,Undefined
2853,2853,Software Developer,Computer/Internet,CGI Veterans Jobs,Albany,Georgia,Usa,Software Developer\n\nShare this\n\nFind simil...,Undefined
6023,6023,Software Developer,computer jobs,GCOM,Albany,New york,United states,"GCOM Software LLC - Albany, NY \nPosition 1: ...",Full Time
7604,7604,Senior Software Developer,Computer-or-internet,Company Info Follow Get job updates from mPowe...,Albany,New york,United states,"Job Function:\n\nCode, test, debug, implement,...",Undefined
9974,9974,Software Developer,Computer-or-internet,Company Info Follow Get job updates from Staff...,Albany,New york,United states,Job Description:\n\nThe selected candidate wil...,Undefined
11364,1364,Environmental Data Scientist,Engineering/Architecture,o'brien & gere engineers,Albany,New york,Usa,Read what people are saying about working here...,Undefined
15443,5450,Environmental Data Scientist,Engineering-or-architecture,o'brien & gere engineers,Albany,New york,United states,"OBG (www.OBG.com), also known as O’Brien & Ger...",Undefined
18601,8608,Principal Data Scientist - Marketing Cloud,math jobs,Oracle,Albany,New york,United states,"Designs, develops and programs methods, proces...",Undefined
